In [ ]:
import sys
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext
from pyspark.streaming.kinesis import KinesisUtils, InitialPositionInStream
from pyspark.sql.types import *

spark = SparkSession.builder\
                    .master("local")\
                    .appName("Structured Streaming - Twitter Sentiment")\
                    .getOrCreate()

In [ ]:

kinesisStreamName = "twitter-data-kinesis" 
kinesisRegion = "us-east-1"

awsAccessKeyId = "" # update the access key
awsSecretKey = "" 

In [ ]:

kinesisDF = spark \
  .readStream \
  .format("kinesis") \
  .option("streamName", kinesisStreamName)\
  .option("region", kinesisRegion) \
  .option("initialPosition", "latest") \
  .option("format", "json") \
  .option("awsAccessKey", awsAccessKeyId)\
  .option("awsSecretKey", awsSecretKey) \
  .option("inferSchema", "true") \
  .load()

df = kinesisDF \
  .writeStream \
  .format("memory") \
  .outputMode("append") \
  .queryName("tweets")  \
  .start()

In [ ]:
df.stop()

--------------------------------------------------------------------------- 
 AttributeError Traceback (most recent call last)
 <command-3229554690368155> in <module> 
 ----> 1 df . stop ( ) 

 /databricks/spark/python/pyspark/sql/dataframe.py in __getattr__ (self, name) 
 1417 if name not in self . columns : 
 1418 raise AttributeError(
 -> 1419 "'%s' object has no attribute '%s'" % (self.__class__.__name__, name))
 1420 jc = self . _jdf . apply ( name ) 
 1421 return Column ( jc ) 

 AttributeError : 'DataFrame' object has no attribute 'stop'

In [ ]:
%sql

select partitionKey, cast(data as string) from tweets;

partitionKey,data
welcomehier,"The pope has joined forces with #Microsoft and #IBM to create a doctrine for ethical #AI and facial recognition. Here's how the Vatican wants to shape it👇 https://t.co/TW653XayEE Microsoft, IBM, AI Fri Nov 27 20:28:00 +0000 2020 0 0 True False 1332420933169143809 AbdulrahmanAlSultan 🇸🇦 welcomehier 2482 SA None"
AlexeyChang,"Machine Learning with Core ML 2 and Swift 5 @udemy ➡️https://t.co/xHqTShMund #MachineLearning #Swift5 #AugmentedReality #DeepLearning #Analytics #DataScience #AI #IoT #IIoT #Python #RStats #CloudComputing #Serverless #Linux #Coding #100DaysofCode #100DaysOfMLCode #udemycoupon https://t.co/qD5q6TLm4a MachineLearning, Swift5, AugmentedReality, DeepLearning, Analytics, DataScience, AI, IoT, IIoT, Python, RStats, CloudComputing, Serverless, Linux, Coding, 100DaysofCode, 100DaysOfMLCode, udemycoupon Fri Nov 27 20:28:04 +0000 2020 0 0 True False 1332420949266886659 Alexey Chang AlexeyChang 146 Russia None"
UW_CSTV,New research: #AI systems lack common sense needed to generate plausible sentences https://t.co/ZDa0IHfkvc via @TechXplore_com AI Fri Nov 27 20:28:07 +0000 2020 0 0 False False 1332420963556843524 CSTV UW_CSTV 190 University of Waterloo None
fintechna,"The latest Breaking Banks Asia! https://t.co/EuUiryRZI1 Thanks to @LeeSlaterSCB @MaximilieMoreau @AlgorithmXLab #fintech #ai fintech, ai Fri Nov 27 20:28:09 +0000 2020 0 0 True False 1332420969680531456 FINTECHNA fintechna 27980 LON | NYC | HKG | SYD | SIN None"
ML_Tweet_Bot,"Machine Learning with Core ML 2 and Swift 5 @udemy ➡️https://t.co/xHqTShMund #MachineLearning #Swift5 #AugmentedReality #DeepLearning #Analytics #DataScience #AI #IoT #IIoT #Python #RStats #CloudComputing #Serverless #Linux #Coding #100DaysofCode #100DaysOfMLCode #udemycoupon https://t.co/qD5q6TLm4a MachineLearning, Swift5, AugmentedReality, DeepLearning, Analytics, DataScience, AI, IoT, IIoT, Python, RStats, CloudComputing, Serverless, Linux, Coding, 100DaysofCode, 100DaysOfMLCode, udemycoupon Fri Nov 27 20:28:09 +0000 2020 0 0 True False 1332420971844788229 Machine Learning Bot ML_Tweet_Bot 4149 None None"
MarkTwa01197091,"Drone taxis are a reality in China! #AI #MachineLearning #Robotics #drone @XHNews @SpirosMargaris @DeepLearn007 @HaroldSinnott @gvalan @Paula_Piccard @ipfconline1 @ShiCooks @diioannid @mvollmer1 @Ym78200 @kalydeoo @Nicochan33 @3itcom @Fabriziobustama https://t.co/Sva8fpJgl1 AI, MachineLearning, Robotics, drone Fri Nov 27 20:28:09 +0000 2020 0 0 True False 1332420972692074496 Mark Twain MarkTwa01197091 10583 None None"
mvollmer1,"6 ways #5G mobile #Broadband will impact the world @wef #AI #MachineLearning #IoT #CES2021 #Automation #4IR #DigitalTransformation #AR #FutureofWork cc @SpirosMargaris @jblefevre60 @Nicochan33 @AudreyDesisto @mvollmer1 @Paula_Piccard https://t.co/lwpBNyGICs 5G, Broadband, AI, MachineLearning, IoT, CES2021, Automation, 4IR, DigitalTransformation, AR, FutureofWork Fri Nov 27 20:28:13 +0000 2020 0 0 True False 1332420989892894720 Dr. Marcell Vollmer #StayHome #StaySafe mvollmer1 65306 Frankfurt, Germany None"
askN0B0DY,"#Trending: UN experts sound alarm over AI-enhanced racial profiling - The Jakarta Post - Jakarta Post #ai #ml #neuralnetworks #deeplearning #machinelearning Read More Here: https://t.co/Cj9HVuiD9Q Trending, ai, ml, neuralnetworks, deeplearning, machinelearning Fri Nov 27 20:28:16 +0000 2020 0 0 True False 1332420999552315394 Nobody askN0B0DY 954 Some Server None"
dcasacuberta,"Looks promising... Fri Nov 27 20:28:18 +0000 2020 0 0 False False 1332421009820045314 David Casacuberta dcasacuberta 2095 Barcelona, Spain None"
SvonFriedeburg,"#GoodNewsFriday: Using artificial intelligence, @NASA scientists mapped 1.8 billion trees to better understand climate change! Fascinating and indicative of the positive impact #AI can have. https://t.co/ua6iVmMZYH GoodNewsFriday, AI Fri Nov 27 20:28:24 +0000 2020 0 0 False True 1332421034054725632 Stephanie von Friedeburg SvonFriedeburg 3124 Washington, DC None"


In [ ]:
tweets = spark.sql("select cast(data as string) from tweets")

In [ ]:
tweets.printSchema()

root
-- data: string (nullable = true)

In [ ]:
tweets.show(5, truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
data |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
The pope has joined forces with #Microsoft and #IBM to create a doctrine for ethical #AI and facial recognition. Here's how the Vatican wants to shape it👇 https://t.co/TW653XayEE	Microsoft, IBM, AI	Fri Nov 27 20:28:00 +0000 2020	0	0	True	False	1332420933169143809	AbdulrahmanAlSultan 🇸🇦	welcomehier	2482	SA	None	

Machine Learning with Core ML 2 and Swift 5 @udemy ➡️https://t.co/xHqTShMund #MachineLearning #Swift5 #AugmentedReality #DeepLearning #Analytics #DataScience #AI #IoT #IIoT #Python #RStats #CloudComputing #Serverless #Linux #Coding #100DaysofCode #100DaysOfMLCode #udemycoupon https://t.co/qD5q6TLm4a	MachineLearning, Swift5, AugmentedReality, DeepLearning, Analytics, DataScience, AI, IoT, IIoT, Python, RStats, CloudComputing, Serverless, Linux, Coding, 100DaysofCode, 100DaysOfMLCode, udemycoupon	Fri Nov 27 20:28:04 +0000 2020	0	0	True	False	1332420949266886659	Alexey Chang	AlexeyChang	146	Russia	None	

New research: #AI systems lack common sense needed to generate plausible sentences https://t.co/ZDa0IHfkvc via @TechXplore_com	AI	Fri Nov 27 20:28:07 +0000 2020	0	0	False	False	1332420963556843524	CSTV	UW_CSTV	190	University of Waterloo	None	

The latest Breaking Banks Asia! https://t.co/EuUiryRZI1 Thanks to @LeeSlaterSCB @MaximilieMoreau @AlgorithmXLab #fintech #ai	fintech, ai	Fri Nov 27 20:28:09 +0000 2020	0	0	True	False	1332420969680531456	FINTECHNA	fintechna	27980	LON | NYC | HKG | SYD | SIN	None	

Machine Learning with Core ML 2 and Swift 5 @udemy ➡️https://t.co/xHqTShMund #MachineLearning #Swift5 #AugmentedReality #DeepLearning #Analytics #DataScience #AI #IoT #IIoT #Python #RStats #CloudComputing #Serverless #Linux #Coding #100DaysofCode #100DaysOfMLCode #udemycoupon https://t.co/qD5q6TLm4a	MachineLearning, Swift5, AugmentedReality, DeepLearning, Analytics, DataScience, AI, IoT, IIoT, Python, RStats, CloudComputing, Serverless, Linux, Coding, 100DaysofCode, 100DaysOfMLCode, udemycoupon	Fri Nov 27 20:28:09 +0000 2020	0	0	True	False	1332420971844788229	Machine Learning Bot	ML_Tweet_Bot	4149	None	None	

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
only showing top 5 rows

In [ ]:
tweets.count()

Out[26]: 978

In [ ]:
tweets_sample = tweets.limit(50)
tweets_sample.cache()
tweets_sample.show(5)

+--------------------+
 data|
+--------------------+
The pope has join...|
Machine Learning ...|
New research: #AI...|
The latest Breaki...|
Machine Learning ...|
+--------------------+
only showing top 5 rows

In [ ]:
tweets_sample.show(5, truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
data |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
The pope has joined forces with #Microsoft and #IBM to create a doctrine for ethical #AI and facial recognition. Here's how the Vatican wants to shape it👇 https://t.co/TW653XayEE	Microsoft, IBM, AI	Fri Nov 27 20:28:00 +0000 2020	0	0	True	False	1332420933169143809	AbdulrahmanAlSultan 🇸🇦	welcomehier	2482	SA	None	

Machine Learning with Core ML 2 and Swift 5 @udemy ➡️https://t.co/xHqTShMund #MachineLearning #Swift5 #AugmentedReality #DeepLearning #Analytics #DataScience #AI #IoT #IIoT #Python #RStats #CloudComputing #Serverless #Linux #Coding #100DaysofCode #100DaysOfMLCode #udemycoupon https://t.co/qD5q6TLm4a	MachineLearning, Swift5, AugmentedReality, DeepLearning, Analytics, DataScience, AI, IoT, IIoT, Python, RStats, CloudComputing, Serverless, Linux, Coding, 100DaysofCode, 100DaysOfMLCode, udemycoupon	Fri Nov 27 20:28:04 +0000 2020	0	0	True	False	1332420949266886659	Alexey Chang	AlexeyChang	146	Russia	None	

New research: #AI systems lack common sense needed to generate plausible sentences https://t.co/ZDa0IHfkvc via @TechXplore_com	AI	Fri Nov 27 20:28:07 +0000 2020	0	0	False	False	1332420963556843524	CSTV	UW_CSTV	190	University of Waterloo	None	

The latest Breaking Banks Asia! https://t.co/EuUiryRZI1 Thanks to @LeeSlaterSCB @MaximilieMoreau @AlgorithmXLab #fintech #ai	fintech, ai	Fri Nov 27 20:28:09 +0000 2020	0	0	True	False	1332420969680531456	FINTECHNA	fintechna	27980	LON | NYC | HKG | SYD | SIN	None	

Machine Learning with Core ML 2 and Swift 5 @udemy ➡️https://t.co/xHqTShMund #MachineLearning #Swift5 #AugmentedReality #DeepLearning #Analytics #DataScience #AI #IoT #IIoT #Python #RStats #CloudComputing #Serverless #Linux #Coding #100DaysofCode #100DaysOfMLCode #udemycoupon https://t.co/qD5q6TLm4a	MachineLearning, Swift5, AugmentedReality, DeepLearning, Analytics, DataScience, AI, IoT, IIoT, Python, RStats, CloudComputing, Serverless, Linux, Coding, 100DaysofCode, 100DaysOfMLCode, udemycoupon	Fri Nov 27 20:28:09 +0000 2020	0	0	True	False	1332420971844788229	Machine Learning Bot	ML_Tweet_Bot	4149	None	None	

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
only showing top 5 rows

Out[7]: 88

In [ ]:
%fs ls /mnt/

path,name,size
dbfs:/mnt/anonymous_telecom/,anonymous_telecom/,0
dbfs:/mnt/aws_logs/,aws_logs/,0
dbfs:/mnt/bikeshare/,bikeshare/,0
dbfs:/mnt/cdr/,cdr/,0
dbfs:/mnt/china_telecom_churn/,china_telecom_churn/,0
dbfs:/mnt/geo/,geo/,0
dbfs:/mnt/movie/,movie/,0
dbfs:/mnt/my_twitter_data_project/,my_twitter_data_project/,0
dbfs:/mnt/orange/,orange/,0
dbfs:/mnt/paysim_fraud/,paysim_fraud/,0


In [ ]:
%fs ls /mnt/my_twitter_data_project/

path,name,size
dbfs:/mnt/my_twitter_data_project/2020/,2020/,0
dbfs:/mnt/my_twitter_data_project/EMR_bootstrap.sh,EMR_bootstrap.sh,231
dbfs:/mnt/my_twitter_data_project/ML_models/,ML_models/,0
dbfs:/mnt/my_twitter_data_project/_metadata,_metadata,0
dbfs:/mnt/my_twitter_data_project/archived/,archived/,0
dbfs:/mnt/my_twitter_data_project/e-C5ILVFSIV1XJG93IA6UOH3E9D/,e-C5ILVFSIV1XJG93IA6UOH3E9D/,0
dbfs:/mnt/my_twitter_data_project/e-DAI1FF74OJ28MX3NGIPULOOE7/,e-DAI1FF74OJ28MX3NGIPULOOE7/,0
dbfs:/mnt/my_twitter_data_project/e-GZ60A1KFQY7FICPF38QRQ33A/,e-GZ60A1KFQY7FICPF38QRQ33A/,0
dbfs:/mnt/my_twitter_data_project/parquet/,parquet/,0
dbfs:/mnt/my_twitter_data_project/sparkLDA.zip,sparkLDA.zip,9752


In [ ]:
sc = spark.sparkContext
sc.addPyFile("dbfs:/mnt/my_twitter_data_project/sparkLDA.zip")

In [ ]:
from sparkLDA.config import n_topics, extra_for_stemmed, seedNum, file_schema
from sparkLDA.utils import show_topics, evaluate
from sparkLDA.processing import preprocess_text

In [ ]:
tweets_sample.schema

Out[16]: StructType(List(StructField(data,StringType,true)))

In [ ]:
tweets_sample.printSchema()

root
-- data: string (nullable = true)

In [ ]:
file_schema_str = StructType([StructField("tweet_text", StringType(), True), 
                          # StructField("hash_tag", ArrayType(StringType(), True), True), 
                          StructField("hash_tag", StringType(), True), 
                          StructField("created_at", StringType(), True), 
                          StructField("retweet_count", StringType(), True), 
                          StructField("favorite_count", StringType(), True), 
                          StructField("retweeted", StringType(), True), 
                          StructField("truncated", StringType(), True), 
                          StructField("id", StringType(), True), 
                          StructField("user_name", StringType(), True), 
                          StructField("screen_name", StringType(), True), 
                          StructField("followers_count", StringType(), True), 
                          StructField("location", StringType(), True), 
                          StructField("geo", StringType(), True),
                          StructField("invalid", StringType(), True)])

In [ ]:
def parse_tweets(raw_records):
  for pdf in raw_records:
    # yield pdf.data.apply(lambda record: record.split("\t"))
    yield pdf.data.str.split(pat="\t", expand=True)
  
tweets_sample.mapInPandas(parse_tweets, schema = file_schema_str).show()

+--------------------+--------------------+--------------------+-------------+--------------+---------+---------+-------------------+--------------------+---------------+---------------+--------------------+----+-------+
 tweet_text| hash_tag| created_at|retweet_count|favorite_count|retweeted|truncated| id| user_name| screen_name|followers_count| location| geo|invalid|
+--------------------+--------------------+--------------------+-------------+--------------+---------+---------+-------------------+--------------------+---------------+---------------+--------------------+----+-------+
The pope has join...| Microsoft, IBM, AI|Fri Nov 27 20:28:...| 0| 0| True| False|1332420933169143809|AbdulrahmanAlSult...| welcomehier| 2482| SA|None| 

Machine Learning ...|MachineLearning, ...|Fri Nov 27 20:28:...| 0| 0| True| False|1332420949266886659| Alexey Chang| AlexeyChang| 146| Russia|None| 

New research: #AI...| AI|Fri Nov 27 20:28:...| 0| 0| False| False|1332420963556843524| CSTV| UW_CSTV| 190|University of Wat...|None| 

The latest Breaki...| fintech, ai|Fri Nov 27 20:28:...| 0| 0| True| False|1332420969680531456| FINTECHNA| fintechna| 27980|LON | NYC | HKG ...|None| 

Machine Learning ...|MachineLearning, ...|Fri Nov 27 20:28:...| 0| 0| True| False|1332420971844788229|Machine Learning Bot| ML_Tweet_Bot| 4149| None|None| 

Drone taxis are a...|AI, MachineLearni...|Fri Nov 27 20:28:...| 0| 0| True| False|1332420972692074496| Mark Twain|MarkTwa01197091| 10583| None|None| 

6 ways #5G mobile...|5G, Broadband, AI...|Fri Nov 27 20:28:...| 0| 0| True| False|1332420989892894720|Dr. Marcell Vollm...| mvollmer1| 65306| Frankfurt, Germany|None| 

#Trending: UN exp...|Trending, ai, ml,...|Fri Nov 27 20:28:...| 0| 0| True| False|1332420999552315394| Nobody| askN0B0DY| 954| Some Server|None| 

 Looks promising...| |Fri Nov 27 20:28:...| 0| 0| False| False|1332421009820045314| David Casacuberta| dcasacuberta| 2095| Barcelona, Spain|None| 

#GoodNewsFriday: ...| GoodNewsFriday, AI|Fri Nov 27 20:28:...| 0| 0| False| True|1332421034054725632|Stephanie von Fri...| SvonFriedeburg| 3124| Washington, DC|None| 

German Startup’s ...|MachineLearning, ...|Fri Nov 27 20:28:...| 0| 0| True| False|1332421049863069704| NourTrabelsi|nourtra00456201| 1184| Wo noch niemand war|None| 

@amazon 5-year-ol...|Alexa, RespectNam...|Fri Nov 27 20:28:...| 0| 0| False| True|1332421049766584322| Human Alexas| HumanAlexas| 839| United States|None| 

The latest The Ar...|deeplearning, mac...|Fri Nov 27 20:28:...| 0| 0| True| False|1332421090459721731| Madhav Kunal| maddykutechno| 214| Edison, NJ|None| 

This cable driven...|Robot, AI, IoT, 4...|Fri Nov 27 20:28:...| 0| 0| True| False|1332421095526461446|Dr. Marcell Vollm...| mvollmer1| 65306| Frankfurt, Germany|None| 

#TechStuff: Opini...|TechStuff, ai, ml...|Fri Nov 27 20:28:...| 0| 0| False| True|1332421163532816387| Chuck Russell|iamChuckRussell| 21454| Camp Hill, PA|None| 

'5⭐️...elements o...|scifi, sciencefic...|Fri Nov 27 20:28:...| 0| 0| True| False|1332421168880525312| Natasha Blackthorne| Nblackthorne| 41642| None|None| 

Organisations tha...| AI, hyperautomation|Fri Nov 27 20:28:...| 0| 0| True| False|1332421171908898817| Nacho H Prado| NachoHPrado| 3304|Madrid, Comunidad...|None| 

#TechStuff: Opini...|TechStuff, ai, ml...|Fri Nov 27 20:28:...| 0| 0| True| False|1332421174526177280| Andrew Morris| andrewmorrisuk| 12134| Bedford|None| 

A beginner's guid...|MachineLearning, ...|Fri Nov 27 20:29:...| 0| 0| True| False|1332421188560281600| SeetaLabs| SeetaLabs| 1013| Italy|None| 

RT/MT: Thanks to ...|HappyFriday, Arti...|Fri Nov 27 20:29:...| 0| 0| False| True|1332421187373240320| Chris Garrod| ChrisGGarrod| 5627|Bermuda, Other Pl...|None| 

+--------------------+--------------------+--------------------+-------------+--------------+---------+---------+-------------------+--------------------+---------------+---------------+--------------------+----+-------+
only showing top 20 rows

process the data in memory

In [ ]:
df = tweets.mapInPandas(parse_tweets, schema = file_schema_str)
df = preprocess_text(df)

In [ ]:
pipelinePath = "dbfs:/mnt/my_twitter_data_project/ML_models/LDA-pipeline-model_Oct_data"
from pyspark.ml import PipelineModel

In [ ]:
savedPipelineModel = PipelineModel.load(pipelinePath)

df_with_topics = savedPipelineModel.transform(df)

In [ ]:
import pyspark.sql.functions as F

from textblob import TextBlob

def get_sentiment_vectorized(texts):
    testimonials = texts.apply(lambda text: TextBlob(text).sentiment.polarity)
    return testimonials
  
getSentiment = F.pandas_udf(get_sentiment_vectorized, FloatType())

In [ ]:
to_array = F.udf(lambda v: v.toArray().tolist(), ArrayType(FloatType()))

df_with_topics = df_with_topics.withColumn("topicDistributionArray", 
                                         to_array("topicDistribution"))

In [ ]:
cols_select = ['tweet_text', 'hash_tag', 'created_at', 'retweet_count', 'favorite_count']
print(cols_select)

['tweet_text', 'hash_tag', 'created_at', 'retweet_count', 'favorite_count']

In [ ]:
df_with_topics = df_with_topics.select(*cols_select, *[(F.col("topicDistributionArray")[i])
                                 .alias("topic_"+str(i)) for i in range(n_topics)])

In [ ]:
df_with_topics.show(5)

+--------------------+--------------------+--------------------+-------------+--------------+-----------+----------+-----------+----------+----------+----------+-----------+-----------+-----------+-----------+
 tweet_text| hash_tag| created_at|retweet_count|favorite_count| topic_0| topic_1| topic_2| topic_3| topic_4| topic_5| topic_6| topic_7| topic_8| topic_9|
+--------------------+--------------------+--------------------+-------------+--------------+-----------+----------+-----------+----------+----------+----------+-----------+-----------+-----------+-----------+
the pope has join...| Microsoft, IBM, AI|Fri Nov 27 20:28:...| 0| 0| 0.10492598|0.07193848| 0.06570499|0.05744596|0.06647186|0.17364535|0.062009085| 0.26428443| 0.07284105|0.060732827|
machine learning ...|MachineLearning, ...|Fri Nov 27 20:28:...| 0| 0|0.089632474|0.08827755| 0.08713678| 0.0833544|0.17070848| 0.1264772|0.087077774| 0.08736985| 0.08575915| 0.09420635|
new research syst...| AI|Fri Nov 27 20:28:...| 0| 0| 0.18605746| 0.067211| 0.07489017|0.18183424|0.11748559|0.07559689|0.070340976|0.069928035|0.089275956| 0.06737969|
the latest breaki...| fintech, ai|Fri Nov 27 20:28:...| 0| 0| 0.09118679|0.09167263|0.106806256| 0.1055957|0.10518496|0.09069894|0.093467824| 0.09957159|0.107262984| 0.10855233|
machine learning ...|MachineLearning, ...|Fri Nov 27 20:28:...| 0| 0|0.089632474|0.08827755| 0.08713678| 0.0833544|0.17070848| 0.1264772|0.087077774| 0.08736985| 0.08575915| 0.09420635|
+--------------------+--------------------+--------------------+-------------+--------------+-----------+----------+-----------+----------+----------+----------+-----------+-----------+-----------+-----------+
only showing top 5 rows

In [ ]:
df_with_topics_sentiment = df_with_topics.withColumn("sentiment", getSentiment(F.col("tweet_text")))\
                            .withColumn("sentiment", F.format_number("sentiment", 3))

In [ ]:
[]

In [ ]:
df_with_topics_sentiment.select("tweet_text", "sentiment", *["topic_"+str(i) for i in range(n_topics)]).show(20, truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------+---------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+
tweet_text |sentiment|topic_0 |topic_1 |topic_2 |topic_3 |topic_4 |topic_5 |topic_6 |topic_7 |topic_8 |topic_9 |
+------------------------------------------------------------------------------------------------------------------------------------------------------------+---------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+
the pope has joined forces with and to create a doctrine for ethical and facial recognition here s how the vatican wants to shape it |0.133 |0.10492598 |0.07193848 |0.06570499 |0.05744596 |0.06647186 |0.17364535 |0.062009085|0.26428443 |0.07284105 |0.060732827|
machine learning with core ml and swift |0.000 |0.089632474|0.08827755 |0.08713678 |0.0833544 |0.17070848 |0.1264772 |0.087077774|0.08736985 |0.08575915 |0.09420635 |
new research systems lack common sense needed to generate plausible sentences via |0.112 |0.18605746 |0.067211 |0.07489017 |0.18183424 |0.11748559 |0.07559689 |0.070340976|0.069928035|0.089275956|0.06737969 |
the latest breaking banks asia thanks to |0.350 |0.09118679 |0.09167263 |0.106806256|0.1055957 |0.10518496 |0.09069894 |0.093467824|0.09957159 |0.107262984|0.10855233 |
machine learning with core ml and swift |0.000 |0.089632474|0.08827755 |0.08713678 |0.0833544 |0.17070848 |0.1264772 |0.087077774|0.08736985 |0.08575915 |0.09420635 |
drone taxis are a reality in china |0.000 |0.08175878 |0.084658064|0.08809326 |0.1828313 |0.08226121 |0.1258502 |0.087564 |0.087808244|0.09262557 |0.086549364|
ways mobile will impact the world cc |0.000 |0.10462556 |0.09600069 |0.096476085|0.10824327 |0.10645242 |0.09645034 |0.09749233 |0.10066631 |0.09615426 |0.09743875 |
un experts sound alarm over ai enhanced racial profiling the jakarta post jakarta post read more here |0.450 |0.064055435|0.3293089 |0.11451924 |0.06630602 |0.061099745|0.07295442 |0.057264805|0.06415868 |0.08237231 |0.087960474|
looks promising |0.200 |0.10754137 |0.09645416 |0.101482704|0.096895345|0.100050874|0.10196502 |0.10181935 |0.09550458 |0.09553902 |0.10274759 |
using artificial intelligence scientists mapped billion trees to better understand climate change fascinating and indicative of the positive impact can have|0.207 |0.105063535|0.102030486|0.081838384|0.10777468 |0.08859508 |0.082259074|0.09974582 |0.07948149 |0.09212145 |0.16109 |
german startup s ai expert roadmap gets k github stars via |0.000 |0.1347757 |0.08917777 |0.079930976|0.08215079 |0.07740592 |0.1382903 |0.10183365 |0.13784581 |0.07544569 |0.08314338 |
year old in the uk |0.100 |0.092689596|0.096837796|0.090412736|0.111408696|0.09388522 |0.09488657 |0.11253832 |0.11730488 |0.09645041 |0.093585774|
the latest the artificial intelligence daily thanks to |0.025 |0.09638157 |0.100420184|0.09969226 |0.10823118 |0.09860085 |0.10035951 |0.0997125 |0.09705115 |0.09878477 |0.10076605 |
this cable driven parallel could be used for different purposes such as product assembling rehabilitation and filmmaking by cc |0.000 |0.06961801 |0.16830279 |0.19091558 |0.06996829 |0.077851065|0.070342466|0.06738052 |0.1405389 |0.08103779 |0.064044595|
opinion how ai can push for equitable access to covid vaccines devex read more here |0.500 |0.076562196|0.078004025|0.083490126|0.08408501 |0.14288819 |0.08613028 |0.11650855 |0.07985518 |0.14639963 |0.106076814|
elements of thrilling mystery suspense dystopian fantasy and science fiction the lenders saga from |0.250 |0.04639203 |0.046183567|0.047057312|0.046319727|0.046210006|0.047138095|0.046209835|0.58145237 |0.046591163|0.04644588 |
organisations that are not applying risk of becoming irrelevant in the market take action today to transform your business |0.017 |0.106172